In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
import time
import os
import math
import random
from scipy.stats import ttest_ind
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [30]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block

In [31]:
def ccm(interaction, block, E_list, tau_list, theta_list, Tp, sample=50, sig=0.05):
    #solver = HistGradientBoostingRegressor() #TRYING DIFFERNT SOLVER TO ENSURE CONVERGENCE



    print(interaction)
    lib = f'1 {len(block)}'
    
    # Get dataframe with two species of interest
    A = interaction[0]; B = interaction[1]
    df = block[[f'{A}(t-0)', f'{B}(t-0)']]
    
    driver = f'{A}(t-0)'    
    default_output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': None,
        'tau': None,
        'theta': None,
        'E_tau_theta_results': None,
        'ccm_value': None,
        'convergence_p_value': None,
        'correlation': None
    }
    
    E_tau_theta_results = pd.DataFrame(columns = ['E', 'tau', 'theta', 'rho'])
    for E, tau, theta in list(product(E_list, tau_list, theta_list)):
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        try:
            c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        except:
            print(f"{interaction} did not converge")
            return default_output
        c = c['predictions'][['Observations', 'Predictions']]
        rho = c.corr().iloc[0,1]
        E_tau_theta_results.loc[len(E_tau_theta_results)] = [E, tau, theta, rho]
    E_tau_theta_results = E_tau_theta_results.apply(pd.to_numeric, errors='coerce')

    # Assign E, tau, and theta to be the optimal E, tau, and theta
    ccm_value = E_tau_theta_results['rho'].max()
    E = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'E'].item())
    tau = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'tau'].item())
    theta = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'theta'].item())
        
    # Get convergence p-value
    try:
        convergence_p_value = get_convergence_p_value(block, sample, A, B, E, Tp, tau, theta)
    except:
        print(f"{interaction} did not converge")
        return default_output

    # Preparing Output
    output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': E,
        'tau': tau,
        'theta': theta,
        'E_tau_theta_results': E_tau_theta_results,
        'ccm_value': ccm_value,
        'convergence_p_value': convergence_p_value,
        'correlation': df.corr().iloc[0,1]
    }

    return output

def get_convergence_p_value(df, sample, A, B, E, Tp, tau, theta):
    # Get convergence p-value for CCM (one-tailed t-test on cross-map values using 20% and 50% library sizes)
    # H0: μ_20% ≥ μ_50%
    # HA: μ_20% < μ_50%
    # If p < 0.05, the 20% library size trials have a rho that is significantly smaller than the 50% library trials  
    
    libsize1 = int(np.ceil(df.shape[0]/5))   # 20% of the full library size
    libsize2 = int(np.ceil(df.shape[0]/2))   # 50% of the full library size
    
    max_iterations = 10 * sample
    
    # Get list of rhos for libsize1
    rhos1 = []; iteration_count = 0
    while len(rhos1) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize1, len(df))
        library = [start - libsize1, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho1 = c.corr().iloc[0,1]
        if not np.isnan(rho1):
            rhos1.append(rho1)
        iteration_count += 1
        
    # Get list of rhos for libsize2
    rhos2 = []; iteration_count = 0
    while len(rhos2) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize2, len(df))
        library = [start - libsize2, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho2 = c.corr().iloc[0,1]
        if not np.isnan(rho2):
            rhos2.append(rho2)
        iteration_count += 1
    
    convergence_t_stat, convergence_p_value = ttest_ind(rhos1, rhos2, alternative='less')
    
    return convergence_p_value


In [32]:
HAB_data = pd.read_csv('Data/orig_block.csv') #LOAD DATA
HAB_data

,serial_day,chlA_.1wk,chlA,nitrate,phosphate,silicate,nitrite,nitrate_phosphate_ratio,N_star,Si_star,...,nitrate_phosphate_ratio_2wk,N_star_2wk,Si_star_2wk,AvgTemp_1wk_2wk,AvgDens_1wk_2wk,SST_2wk,WIND_SPD_2wk,U_WIND_2wk,PRCP_2wkavg_2wk,AWND_2wkavg_2wk
0,725662,0.920,0.430,2.520,0.295,2.355,0.020,8.542373,-2.200,2.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,725669,0.615,0.920,2.110,0.305,2.605,0.115,6.918033,-2.770,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,725676,0.655,0.615,1.915,0.370,0.450,0.020,5.175676,-4.005,0.08,...,8.542373,-2.200,2.06,19.125000,1023.8997,19.118428,3.219574,0.398250,25.214286,29.000000
3,725683,0.410,0.655,0.155,0.330,1.500,0.020,0.469697,-5.125,1.17,...,6.918033,-2.770,2.30,18.564286,1024.0495,19.229762,3.243543,0.701107,0.000000,30.428571
4,725690,0.325,0.410,0.200,0.320,2.000,0.030,0.625000,-4.920,1.68,...,5.175676,-4.005,0.08,17.721429,1024.2147,19.083670,2.679270,0.652186,0.000000,29.357143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,734926,1.995,1.790,0.940,0.280,4.930,0.120,3.357143,-3.540,4.65,...,2.464286,-3.790,2.40,15.253571,1024.7554,14.476527,3.936310,0.646581,6.000000,19.357143
1837,734933,8.210,1.995,4.360,0.450,7.690,0.270,9.688889,-2.840,7.24,...,5.166667,-3.250,4.29,14.600000,1024.8883,14.179503,4.900595,1.848541,9.428571,25.642857
1838,734940,2.775,8.210,1.490,0.310,4.710,0.110,4.806452,-3.470,4.40,...,3.357143,-3.540,4.65,14.828571,1024.8475,14.007738,4.063095,0.773389,15.428571,26.428571
1839,734947,2.410,2.775,3.760,0.600,6.280,0.230,6.266667,-5.840,5.68,...,9.688889,-2.840,7.24,14.316667,1024.9582,13.920575,4.294345,0.868929,6.928571,24.214286


In [33]:
#HAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0)#.iloc[304:612] RANGE w/o missing values
HAB_data = HAB_data.reset_index()
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]
# HAB_data = HAB_data.set_index('time')
target = 'chlA' #change target if needed

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))

HAB_data

,AWND_2wkavg,AWND_2wkavg_1wk,AWND_2wkavg_2wk,AvgDens_1wk,AvgDens_1wk_1wk,AvgDens_1wk_2wk,AvgTemp_1wk,AvgTemp_1wk_1wk,AvgTemp_1wk_2wk,N_star,...,nitrite,nitrite_1wk,nitrite_2wk,phosphate,phosphate_1wk,phosphate_2wk,serial_day,silicate,silicate_1wk,silicate_2wk
0,29.000000,NaN,NaN,1023.8997,NaN,NaN,19.125000,NaN,NaN,-2.200,...,0.020,NaN,NaN,0.295,NaN,NaN,725662,2.355,NaN,NaN
1,30.428571,29.000000,NaN,1024.0495,1023.8997,NaN,18.564286,19.125000,NaN,-2.770,...,0.115,0.020,NaN,0.305,0.295,NaN,725669,2.605,2.355,NaN
2,29.357143,30.428571,29.000000,1024.2147,1024.0495,1023.8997,17.721429,18.564286,19.125000,-4.005,...,0.020,0.115,0.020,0.370,0.305,0.295,725676,0.450,2.605,2.355
3,28.214286,29.357143,30.428571,1024.1421,1024.2147,1024.0495,18.135714,17.721429,18.564286,-5.125,...,0.020,0.020,0.115,0.330,0.370,0.305,725683,1.500,0.450,2.605
4,27.214286,28.214286,29.357143,1024.2600,1024.1421,1024.2147,17.650000,18.135714,17.721429,-4.920,...,0.030,0.020,0.020,0.320,0.330,0.370,725690,2.000,1.500,0.450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,26.428571,25.642857,19.357143,1024.8475,1024.8883,1024.7554,14.828571,14.600000,15.253571,-3.540,...,0.120,0.120,0.060,0.280,0.300,0.280,734926,4.930,4.590,2.680
1837,24.214286,26.428571,25.642857,1024.9582,1024.8475,1024.8883,14.316667,14.828571,14.600000,-2.840,...,0.270,0.120,0.120,0.450,0.280,0.300,734933,7.690,4.930,4.590
1838,24.000000,24.214286,26.428571,1025.1464,1024.9582,1024.8475,13.807143,14.316667,14.828571,-3.470,...,0.110,0.270,0.120,0.310,0.450,0.280,734940,4.710,7.690,4.930
1839,33.142857,24.000000,24.214286,1025.1721,1025.1464,1024.9582,13.735714,13.807143,14.316667,-5.840,...,0.230,0.110,0.270,0.600,0.310,0.450,734947,6.280,4.710,7.690


In [34]:
HAB_data.columns

#dropping columns with 1wk or 2wk
cols_to_drop = [col for col in HAB_data.columns if 
    '1wk' in col or
    '_2wk' in col]

HAB_data = HAB_data.drop(columns = cols_to_drop)
HAB_data

,N_star,SST,Si_star,U_WIND,WIND_SPD,chlA,index,nitrate,nitrate_phosphate_ratio,nitrite,phosphate,serial_day,silicate
0,-2.200,19.118428,2.06,0.398250,3.219574,0.430,0,2.520,8.542373,0.020,0.295,725662,2.355
1,-2.770,19.229762,2.30,0.701107,3.243543,0.920,1,2.110,6.918033,0.115,0.305,725669,2.605
2,-4.005,19.083670,0.08,0.652186,2.679270,0.615,2,1.915,5.175676,0.020,0.370,725676,0.450
3,-5.125,18.651255,1.17,0.735618,2.754969,0.655,3,0.155,0.469697,0.020,0.330,725683,1.500
4,-4.920,18.196674,1.68,0.583428,2.934498,0.410,4,0.200,0.625000,0.030,0.320,725690,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,-3.540,14.007738,4.65,0.773389,4.063095,1.790,1836,0.940,3.357143,0.120,0.280,734926,4.930
1837,-2.840,13.920575,7.24,0.868929,4.294345,1.995,1837,4.360,9.688889,0.270,0.450,734933,7.690
1838,-3.470,13.516654,4.40,1.807142,4.558411,8.210,1838,1.490,4.806452,0.110,0.310,734940,4.710
1839,-5.840,13.329387,5.68,0.621545,4.609058,2.775,1839,3.760,6.266667,0.230,0.600,734947,6.280


In [35]:
print(HAB_data.isna().sum())

N_star                     0
SST                        0
Si_star                    0
U_WIND                     0
WIND_SPD                   0
chlA                       0
index                      0
nitrate                    0
nitrate_phosphate_ratio    0
nitrite                    0
phosphate                  0
serial_day                 0
silicate                   0
dtype: int64


In [36]:
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
HAB_data = HAB_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(HAB_data)
# HAB_data = Imputer.transform(HAB_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
HAB_data

,N_star,SST,Si_star,U_WIND,WIND_SPD,chlA,index,nitrate,nitrate_phosphate_ratio,nitrite,phosphate,serial_day,silicate
0,-2.200,19.118428,2.06,0.398250,3.219574,0.430,0,2.520,8.542373,0.020,0.295,725662,2.355
1,-2.770,19.229762,2.30,0.701107,3.243543,0.920,1,2.110,6.918033,0.115,0.305,725669,2.605
2,-4.005,19.083670,0.08,0.652186,2.679270,0.615,2,1.915,5.175676,0.020,0.370,725676,0.450
3,-5.125,18.651255,1.17,0.735618,2.754969,0.655,3,0.155,0.469697,0.020,0.330,725683,1.500
4,-4.920,18.196674,1.68,0.583428,2.934498,0.410,4,0.200,0.625000,0.030,0.320,725690,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,-3.540,14.007738,4.65,0.773389,4.063095,1.790,1836,0.940,3.357143,0.120,0.280,734926,4.930
1837,-2.840,13.920575,7.24,0.868929,4.294345,1.995,1837,4.360,9.688889,0.270,0.450,734933,7.690
1838,-3.470,13.516654,4.40,1.807142,4.558411,8.210,1838,1.490,4.806452,0.110,0.310,734940,4.710
1839,-5.840,13.329387,5.68,0.621545,4.609058,2.775,1839,3.760,6.266667,0.230,0.600,734947,6.280


In [37]:
print(HAB_data.isna().sum())

N_star                     0
SST                        0
Si_star                    0
U_WIND                     0
WIND_SPD                   0
chlA                       0
index                      0
nitrate                    0
nitrate_phosphate_ratio    0
nitrite                    0
phosphate                  0
serial_day                 0
silicate                   0
dtype: int64


In [38]:
#STANDARDIZE
#LEFT OFF HERE THERE IS AN ERROR
'''
scaler = MinMaxScaler()
columns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', 
           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', 
           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', 
           'Silicate', 'WSPD']
HAB_data[columns] = scaler.fit_transform(HAB_data[columns])
HAB_data
'''

"\nscaler = MinMaxScaler()\ncolumns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', \n           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', \n           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', \n           'Silicate', 'WSPD']\nHAB_data[columns] = scaler.fit_transform(HAB_data[columns])\nHAB_data\n"

In [39]:
#HAB_data.drop(columns=[]) # FOR MELISSA CARTER DATA
block = get_block(HAB_data, num_lags=50, tau=1)
block

,N_star(t-0),SST(t-0),Si_star(t-0),U_WIND(t-0),WIND_SPD(t-0),chlA(t-0),index(t-0),nitrate(t-0),nitrate_phosphate_ratio(t-0),nitrite(t-0),...,U_WIND(t-50),WIND_SPD(t-50),chlA(t-50),index(t-50),nitrate(t-50),nitrate_phosphate_ratio(t-50),nitrite(t-50),phosphate(t-50),serial_day(t-50),silicate(t-50)
0,-2.200,19.118428,2.06,0.398250,3.219574,0.430,0,2.520,8.542373,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2.770,19.229762,2.30,0.701107,3.243543,0.920,1,2.110,6.918033,0.115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-4.005,19.083670,0.08,0.652186,2.679270,0.615,2,1.915,5.175676,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-5.125,18.651255,1.17,0.735618,2.754969,0.655,3,0.155,0.469697,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-4.920,18.196674,1.68,0.583428,2.934498,0.410,4,0.200,0.625000,0.030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,-3.540,14.007738,4.65,0.773389,4.063095,1.790,1836,0.940,3.357143,0.120,...,0.422999,3.262202,1.640,1786.0,0.80,1.951220,0.04,0.41,734551.0,0.04
1837,-2.840,13.920575,7.24,0.868929,4.294345,1.995,1837,4.360,9.688889,0.270,...,-0.254430,4.166964,1.775,1787.0,0.19,0.678571,0.01,0.28,734558.0,0.01
1838,-3.470,13.516654,4.40,1.807142,4.558411,8.210,1838,1.490,4.806452,0.110,...,-0.054776,4.640476,1.835,1788.0,0.34,1.133333,0.02,0.30,734565.0,0.02
1839,-5.840,13.329387,5.68,0.621545,4.609058,2.775,1839,3.760,6.266667,0.230,...,1.236447,4.807660,4.520,1789.0,0.26,2.000000,0.01,0.13,734572.0,0.01


In [40]:
E_list = range(2,13)
tau_list = [-1,-2,-3] #can try more taus
theta_list = [0,0.1,0.5,1,2,3,4,5,6,7,8,9]
Tp = 0
exclusion_radius = 0

all_ccm_results = pd.DataFrame()
interactions = list(permutations(HAB_data.columns.tolist(),2))
print(interactions)
target_interactions = [pair for pair in interactions if target in pair]

interaction = target_interactions[0]
print(f'There are {len(target_interactions)} interactions')

results = Parallel(n_jobs=8)(
    delayed(ccm)(interaction, block, E_list, tau_list, theta_list, Tp) for interaction in target_interactions)
results_df = pd.DataFrame(results)

[('N_star', 'SST'), ('N_star', 'Si_star'), ('N_star', 'U_WIND'), ('N_star', 'WIND_SPD'), ('N_star', 'chlA'), ('N_star', 'index'), ('N_star', 'nitrate'), ('N_star', 'nitrate_phosphate_ratio'), ('N_star', 'nitrite'), ('N_star', 'phosphate'), ('N_star', 'serial_day'), ('N_star', 'silicate'), ('SST', 'N_star'), ('SST', 'Si_star'), ('SST', 'U_WIND'), ('SST', 'WIND_SPD'), ('SST', 'chlA'), ('SST', 'index'), ('SST', 'nitrate'), ('SST', 'nitrate_phosphate_ratio'), ('SST', 'nitrite'), ('SST', 'phosphate'), ('SST', 'serial_day'), ('SST', 'silicate'), ('Si_star', 'N_star'), ('Si_star', 'SST'), ('Si_star', 'U_WIND'), ('Si_star', 'WIND_SPD'), ('Si_star', 'chlA'), ('Si_star', 'index'), ('Si_star', 'nitrate'), ('Si_star', 'nitrate_phosphate_ratio'), ('Si_star', 'nitrite'), ('Si_star', 'phosphate'), ('Si_star', 'serial_day'), ('Si_star', 'silicate'), ('U_WIND', 'N_star'), ('U_WIND', 'SST'), ('U_WIND', 'Si_star'), ('U_WIND', 'WIND_SPD'), ('U_WIND', 'chlA'), ('U_WIND', 'index'), ('U_WIND', 'nitrate'), ('

In [41]:
# Get CCM results that show convergence (convergence p-value < 0.05)
results_df = pd.DataFrame(results)
ccm_cutoff = -1

significant_results = results_df[results_df.convergence_p_value<0.20]
significant_results = significant_results.sort_values(by='ccm_value', ascending=False)
significant_results = significant_results[['target (driver)', 'lib (driven)', 'E', 'tau', 'theta', 'ccm_value']].reset_index(drop=True)

display(significant_results[significant_results.ccm_value>ccm_cutoff])

# Choose system variables where the CCM value to or from the target is > ccm_cutoff
system_variables = significant_results[significant_results.ccm_value > ccm_cutoff]
system_variables = system_variables[['target (driver)', 'lib (driven)']].values.flatten().tolist()
system_variables = list(set(system_variables))
print('system variables: ')
display(sorted(system_variables))

,target (driver),lib (driven),E,tau,theta,ccm_value
0,index,chlA,12,-3,4,0.587284
1,serial_day,chlA,12,-3,4,0.564261
2,chlA,SST,8,-2,9,0.543592
3,chlA,U_WIND,4,-3,9,0.500872
4,chlA,WIND_SPD,10,-1,8,0.461403
5,N_star,chlA,10,-2,3,0.423217
6,silicate,chlA,12,-2,2,0.367219
7,chlA,nitrite,9,-3,9,0.351162
8,phosphate,chlA,12,-1,4,0.299121
9,chlA,phosphate,2,-1,9,0.256421


system variables: 


['N_star',
 'SST',
 'U_WIND',
 'WIND_SPD',
 'chlA',
 'index',
 'nitrate',
 'nitrite',
 'phosphate',
 'serial_day',
 'silicate']

In [42]:
results_df[results_df.convergence_p_value<0.1].sort_values(by='convergence_p_value', ascending=False)

,target (driver),lib (driven),E,tau,theta,E_tau_theta_results,ccm_value,convergence_p_value,correlation
0,N_star,chlA,10,-2,3,E tau theta rho 0 2.0 -1.0...,0.423217,9.829972e-02,0.137084
18,nitrate,chlA,12,-1,5,E tau theta rho 0 2.0 -1.0...,0.196728,3.394427e-02,-0.005529
23,silicate,chlA,12,-2,2,E tau theta rho 0 2.0 -1.0...,0.367219,3.196271e-02,0.244880
22,serial_day,chlA,12,-3,4,E tau theta rho 0 2.0 -1.0...,0.564261,1.819601e-03,0.107600
17,index,chlA,12,-3,4,E tau theta rho 0 2.0 -1.0...,0.587284,1.241870e-05,0.124132
21,phosphate,chlA,12,-1,4,E tau theta rho 0 2.0 -1.0...,0.299121,1.199400e-05,-0.106336
14,chlA,phosphate,2,-1,9,E tau theta rho 0 2.0 -1.0...,0.256421,1.013031e-05,-0.106336
6,chlA,SST,8,-2,9,E tau theta rho 0 2.0 -1.0...,0.543592,2.625469e-07,-0.121944
5,chlA,N_star,5,-2,3,E tau theta rho 0 2.0 -1.0...,0.214797,1.660604e-07,0.137084
9,chlA,WIND_SPD,10,-1,8,E tau theta rho 0 2.0 -1.0...,0.461403,5.302378e-08,0.067154
